In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [2]:
from gpt_converter import gptkeys
import os

os.environ["OPENAI_API_KEY"] = gptkeys.getone()

In [3]:
import os

# LANGSMITH 프로젝트 정보 설정
# LangSmith API KEY 가 없다면 건너뛰기
# https://smith.langchain.com/
os.environ["LANGCHAIN_PROJECT"] = "LCEL"

In [88]:
from langchain.prompts import PromptTemplate

# 프롬프트를 생성합니다
prompt = PromptTemplate.from_template("{num} 의 10배는?")
prompt

PromptTemplate(input_variables=['num'], template='{num} 의 10배는?')

In [89]:
from langchain_openai import ChatOpenAI

# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")

In [90]:
# chain 을 실행합니다.
chain.invoke({"num": 5})

AIMessage(content='5의 10배는 50입니다.')

## 데이터를 효과적으로 전달하는 방법

RunnablePassthrough는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다. 이는 주로 RunnableParallel과 함께 사용되어 맵에서 새 키에 데이터를 할당하는 데 사용됩니다.

RunnablePassthrough()가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.

RunnablePassthrough가 assign (`RunnablePassthrough.assign(...)`)과 함께 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.


In [91]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
)

## RunnablePassthrough


In [93]:
# chain 을 실행합니다.
chain.invoke({"num": 2})

AIMessage(content='2의 10배는 20입니다.')

아래는 `RunnablePassthrough` 를 사용한 예제입니다.


In [94]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.')

다음은 `RunnablePassthrough.assign()` 을 사용하는 경우와 비교한 결과입니다.


In [95]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

In [96]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## RunnableParallel


In [98]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

Chain 도 RunnableParallel 적용할 수 있습니다.


In [112]:
chain1 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 10배는?\n답변(결과만): ")
    | ChatOpenAI()
)
chain2 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 1/10배는?\n답변(결과만): ")
    | ChatOpenAI()
)

In [113]:
combined_chain = RunnableParallel(a=chain1, b=chain2)
combined_chain.invoke({"num": 10})

{'a': AIMessage(content='100'), 'b': AIMessage(content='1')}

## RunnableLambda

RunnableLambda 를 사용하여 사용자 정의 함수를 맵핑할 수 있습니다.


In [115]:
def extra(x):
    ext = x["extra"]
    return int(ext["num"]) * int(ext["mult"])

In [116]:
(runnable | RunnableLambda(extra)).invoke({"num": 3})

27